IMPORT MODULES

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

---

In [2]:
test_features = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [3]:
training_data = pd.read_csv('kc_house_data_train.csv', index_col=0)

In [4]:
training_data['has_basement'] = np.where(training_data['sqft_basement'] == 0, 0, 1)

In [5]:
categorical = ['floors', 'waterfront', 'view', 'condition', 'grade', 'zipcode', 'has_basement']
continuous = ['sqft_living', 'bedrooms', 'bathrooms', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']
price = training_data['price']

In [6]:
#numbers to measure against when getting RMSE or MSE

print('price min:', price.min())
print('price max:', price.max())
print('price mean:', price.mean())
print('price median:', price.median())

price min: 75000.0
price max: 7700000.0
price mean: 540739.5486408329
price median: 450000.0


In [7]:
cont_data = training_data[continuous]
cat_data = training_data[categorical]

In [8]:
cont_data.corr() 

,sqft_living,bedrooms,bathrooms,sqft_lot,sqft_above,sqft_basement,sqft_living15,sqft_lot15
sqft_living,1.000000,0.570428,0.755270,0.166810,0.876696,0.435529,0.755066,0.179570
bedrooms,0.570428,1.000000,0.507867,0.029375,0.472492,0.300085,0.387589,0.026665
bathrooms,0.755270,0.507867,1.000000,0.085470,0.686853,0.282700,0.568080,0.086499
sqft_lot,0.166810,0.029375,0.085470,1.000000,0.180755,0.008065,0.136959,0.701875
sqft_above,0.876696,0.472492,0.686853,0.180755,1.000000,-0.051199,0.731077,0.193874
sqft_basement,0.435529,0.300085,0.282700,0.008065,-0.051199,1.000000,0.199521,0.010007
sqft_living15,0.755066,0.387589,0.568080,0.136959,0.731077,0.199521,1.000000,0.178533
sqft_lot15,0.179570,0.026665,0.086499,0.701875,0.193874,0.010007,0.178533,1.000000


In [9]:
cat_data.corr() 

,floors,waterfront,view,condition,grade,zipcode,has_basement
floors,1.000000,0.025132,0.025883,-0.259230,0.458984,-0.060560,-0.260333
waterfront,0.025132,1.000000,0.407565,0.009608,0.092234,0.034282,0.039484
view,0.025883,0.407565,1.000000,0.045060,0.249554,0.089475,0.187475
condition,-0.259230,0.009608,0.045060,1.000000,-0.142066,0.007068,0.131918
grade,0.458984,0.092234,0.249554,-0.142066,1.000000,-0.184946,0.048776
zipcode,-0.060560,0.034282,0.089475,0.007068,-0.184946,1.000000,0.167239
has_basement,-0.260333,0.039484,0.187475,0.131918,0.048776,0.167239,1.000000


In [10]:
#choosing 3 cont variables + 3 categorical vars
features = ['sqft_living', 'bedrooms', 'bathrooms', 'view_grade', 'waterfront', 'zipcode', 'has_basement']

In [11]:
conditions = [training_data['bathrooms'] <= 1, training_data['bathrooms'] <= 1.75,
              training_data['bathrooms'] <= 2, training_data['bathrooms'] <= 2.75,
              training_data['bathrooms'] <= 3, training_data['bathrooms'] <= 3.75,
              training_data['bathrooms'] <= 4, training_data['bathrooms'] <= 4.75]
choices = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5]

training_data['bathrooms'] = np.select(conditions, choices, 5)

In [12]:
#feature engineering

#if a house has 1 or more views, it's a 1 otherwise a 0
# training_data['view_bin'] = np.where(training_data['view'].values >= 1, 1,0)
training_data['bedrooms'] = np.where((training_data.bedrooms > 6),6,training_data['bedrooms'])
training_data['view_grade'] = cat_data['view'] + cat_data['grade']

In [13]:
# Continuous features to scale 
#living = training_data['sqft_living']
logliving = np.log(training_data['sqft_living'])
bed = training_data['bedrooms']
bath = training_data['bathrooms']

In [14]:
# Scale the features
scaler = StandardScaler()
#scaled_living = scaler.fit_transform(training_data[['sqft_living']])
scaled_livinglog = scaler.fit_transform(np.log(training_data[['sqft_living']]))
scaled_bed = scaler.fit_transform(training_data[['bedrooms']])
scaled_bath = scaler.fit_transform(training_data[['bathrooms']])

In [15]:
# Create a dataframe with scaled features
data_fin = pd.DataFrame([])
data_fin['living'] = scaled_livinglog.flatten()
data_fin['bed'] = scaled_bed.flatten()
data_fin['bath'] = scaled_bath.flatten()

In [16]:
# Create dummies for categorical features
view_grade_dummies = pd.get_dummies(training_data['view_grade'], prefix='view_grade', drop_first=True)
zip_dummies = pd.get_dummies(training_data['zipcode'], prefix='zipcode', drop_first=True)
water = training_data['waterfront']
base = training_data['has_basement']

In [17]:
concat_vg_dums = pd.concat([price, view_grade_dummies], axis=1)

In [18]:
concat_zip_dums = pd.concat([price, zip_dummies], axis=1)

In [19]:
view_grade_dummies.describe()

,view_grade_3,view_grade_4,view_grade_5,view_grade_6,view_grade_7,view_grade_8,view_grade_9,view_grade_10,view_grade_11,view_grade_12,view_grade_13,view_grade_14,view_grade_15,view_grade_16,view_grade_17
count,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000
mean,0.000116,0.001157,0.010526,0.091787,0.395662,0.255639,0.117178,0.062233,0.029265,0.016773,0.010064,0.005899,0.002198,0.001272,0.000174
std,0.010755,0.033992,0.102059,0.288734,0.489007,0.436232,0.321641,0.241584,0.168555,0.128422,0.099814,0.076583,0.046831,0.035649,0.013172
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
f = 'price~' + '+'.join(zip_dummies.columns)
model_untouched = ols(formula=f, data=concat_zip_dums).fit()
model_untouched.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.399
Method:                 Least Squares   F-statistic:                     167.4
Date:                Mon, 04 May 2020   Prob (F-statistic):               0.00
Time:                        15:09:37   Log-Likelihood:            -2.4193e+05
No. Observations:               17290   AIC:                         4.840e+05
Df Residuals:                   17220   BIC:                         4.845e+05
Df Model:                          69                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       2.82e+05   1.72e+04     16.421      0.000    2.48e+05    3.16e+05
zipcode_98002 -4.971e+04   2.88e+04     -1.727      0.084   -1.06e+05    6701.740
zipcode_98003  8763.9215   2.59e+04      0.339      0.735   -4.19e+04    5.95e+04
zipcode_98004  1.115e+06   2.55e+04     43.787      0.000    1.06e+06    1.16e+06
zipcode_98005  5.268e+05   2.99e+04     17.629      0.000    4.68e+05    5.85e+05
zipcode_98006  5.675e+05   2.26e+04     25.068      0.000    5.23e+05    6.12e+05
zipcode_98007  3.156e+05   3.13e+04     10.073      0.000    2.54e+05    3.77e+05
zipcode_98008  3.588e+05   2.58e+04     13.907      0.000    3.08e+05    4.09e+05
zipcode_98010  1.429e+05   3.78e+04      3.783      0.000    6.88e+04    2.17e+05
zipcode_98011  2.086e+05   2.84e+04      7.337      0.000    1.53e+05    2.64e+05
zipcode_98014  1.742e+05   3.35e+04      5.196      0.000    1.08e+05     2.4e+05
zipcode_98019  1.464e+05   2.95e+04      4.957      0.000    8.85e+04    2.04e+05
zipcode_98022  2.677e+04   2.72e+04      0.985      0.324   -2.65e+04       8e+04
zipcode_98023  7573.8820   2.23e+04      0.339      0.735   -3.62e+04    5.14e+04
zipcode_98024   3.09e+05   3.98e+04      7.765      0.000    2.31e+05    3.87e+05
zipcode_98027  3.253e+05   2.34e+04     13.907      0.000    2.79e+05    3.71e+05
zipcode_98028  1.864e+05    2.6e+04      7.163      0.000    1.35e+05    2.37e+05
zipcode_98029  3.337e+05   2.51e+04     13.295      0.000    2.84e+05    3.83e+05
zipcode_98030   1.44e+04   2.67e+04      0.540      0.589   -3.79e+04    6.67e+04
zipcode_98031   1.86e+04   2.62e+04      0.711      0.477   -3.27e+04    6.99e+04
zipcode_98032  -3.04e+04   3.33e+04     -0.913      0.361   -9.56e+04    3.48e+04
zipcode_98033  5.207e+05    2.3e+04     22.601      0.000    4.76e+05    5.66e+05
zipcode_98034  2.419e+05   2.21e+04     10.955      0.000    1.99e+05    2.85e+05
zipcode_98038  8.642e+04   2.19e+04      3.949      0.000    4.35e+04    1.29e+05
zipcode_98039  1.921e+06   4.78e+04     40.147      0.000    1.83e+06    2.01e+06
zipcode_98040   9.01e+05   2.57e+04     35.055      0.000    8.51e+05    9.51e+05
zipcode_98042  2.717e+04   2.22e+04      1.224      0.221   -1.63e+04    7.07e+04
zipcode_98045  1.493e+05   2.78e+04      5.378      0.000    9.49e+04    2.04e+05
zipcode_98052  3.641e+05   2.17e+04     16.764      0.000    3.21e+05    4.07e+05
zipcode_98053  3.953e+05   2.35e+04     16.805      0.000    3.49e+05    4.41e+05
zipcode_98055  2.485e+04   2.63e+04      0.944      0.345   -2.68e+04    7.65e+04
zipcode_98056  1.415e+05   2.33e+04      6.066      0.000    9.58e+04    1.87e+05
zipcode_98058  7.237e+04   2.26e+04      3.197      0.001     2.8e+04    1.17e+05
zipcode_98059  2.124e+05   2.28e+04      9.318      0.000    1.68e+05    2.57e+05
zipcode_98065  2.452e+05   2.51e+04      9.760   

In [21]:
zip_dummies.drop(columns=['zipcode_98003', 'zipcode_98022', 'zipcode_98023', 'zipcode_98031', 'zipcode_98032', 
                          'zipcode_98042', 'zipcode_98055', 'zipcode_98106', 'zipcode_98168', 'zipcode_98198',
                          'zipcode_98002', 'zipcode_98038'],
                inplace=True)

In [22]:
training_data.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'has_basement',
       'view_grade'],
      dtype='object')

In [23]:
#drop the grade dummies with high p-values
view_grade_dummies.drop(columns=['view_grade_3','view_grade_4', 'view_grade_5', 'view_grade_6',
                                 'view_grade_7', 'view_grade_8'], 
                        inplace=True)

In [24]:
#separate the result
price = training_data['price']

In [25]:
# Concat with dataframe 
data_fin = pd.concat([price, data_fin, water, base, view_grade_dummies, zip_dummies], axis=1)

In [26]:
data_fin.columns[0:20], data_fin.columns[20:40], data_fin.columns[40:60]

(Index(['price', 'living', 'bed', 'bath', 'waterfront', 'has_basement',
        'view_grade_9', 'view_grade_10', 'view_grade_11', 'view_grade_12',
        'view_grade_13', 'view_grade_14', 'view_grade_15', 'view_grade_16',
        'view_grade_17', 'zipcode_98004', 'zipcode_98005', 'zipcode_98006',
        'zipcode_98007', 'zipcode_98008'],
       dtype='object'),
 Index(['zipcode_98010', 'zipcode_98011', 'zipcode_98014', 'zipcode_98019',
        'zipcode_98024', 'zipcode_98027', 'zipcode_98028', 'zipcode_98029',
        'zipcode_98030', 'zipcode_98033', 'zipcode_98034', 'zipcode_98039',
        'zipcode_98040', 'zipcode_98045', 'zipcode_98052', 'zipcode_98053',
        'zipcode_98056', 'zipcode_98058', 'zipcode_98059', 'zipcode_98065'],
       dtype='object'),
 Index(['zipcode_98070', 'zipcode_98072', 'zipcode_98074', 'zipcode_98075',
        'zipcode_98077', 'zipcode_98092', 'zipcode_98102', 'zipcode_98103',
        'zipcode_98105', 'zipcode_98107', 'zipcode_98108', 'zipcode_98109',
 

In [27]:
# Create model
outcome = 'price'
predictors = data_fin.drop('price', axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum
model = ols(formula=formula, data=data_fin).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.793
Method:                 Least Squares   F-statistic:                     934.1
Date:                Mon, 04 May 2020   Prob (F-statistic):               0.00
Time:                        15:09:45   Log-Likelihood:            -2.3271e+05
No. Observations:               17290   AIC:                         4.656e+05
Df Residuals:                   17218   BIC:                         4.661e+05
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      3.194e+05   3196.234     99.946      0.000    3.13e+05    3.26e+05
living         1.229e+05   2556.202     48.076      0.000    1.18e+05    1.28e+05
bed           -7460.6456   1746.474     -4.272      0.000   -1.09e+04   -4037.378
bath            2.21e+04   1970.306     11.214      0.000    1.82e+04     2.6e+04
waterfront     4.521e+05   1.62e+04     27.867      0.000     4.2e+05    4.84e+05
has_basement   -3.47e+04   2981.401    -11.638      0.000   -4.05e+04   -2.89e+04
view_grade_9   7.851e+04   4543.981     17.279      0.000    6.96e+04    8.74e+04
view_grade_10  1.676e+05   5999.576     27.943      0.000    1.56e+05    1.79e+05
view_grade_11  2.817e+05   8269.899     34.064      0.000    2.65e+05    2.98e+05
view_grade_12  4.062e+05   1.08e+04     37.612      0.000    3.85e+05    4.27e+05
view_grade_13  6.007e+05   1.36e+04     44.018      0.000    5.74e+05    6.27e+05
view_grade_14  8.236e+05   1.78e+04     46.385      0.000    7.89e+05    8.58e+05
view_grade_15  9.958e+05   2.88e+04     34.552      0.000    9.39e+05    1.05e+06
view_grade_16  1.981e+06   3.77e+04     52.465      0.000    1.91e+06    2.05e+06
view_grade_17  3.168e+06    9.9e+04     31.999      0.000    2.97e+06    3.36e+06
zipcode_98004  8.446e+05   1.15e+04     73.147      0.000    8.22e+05    8.67e+05
zipcode_98005  3.462e+05   1.47e+04     23.513      0.000    3.17e+05    3.75e+05
zipcode_98006  2.725e+05   9323.234     29.227      0.000    2.54e+05    2.91e+05
zipcode_98007   2.49e+05   1.57e+04     15.881      0.000    2.18e+05     2.8e+05
zipcode_98008  2.444e+05   1.17e+04     20.863      0.000    2.21e+05    2.67e+05
zipcode_98010  6.368e+04      2e+04      3.188      0.001    2.45e+04    1.03e+05
zipcode_98011  1.225e+05   1.36e+04      8.977      0.000    9.57e+04    1.49e+05
zipcode_98014  1.048e+05   1.72e+04      6.100      0.000    7.11e+04    1.38e+05
zipcode_98019  7.197e+04   1.44e+04      4.984      0.000    4.37e+04       1e+05
zipcode_98024  1.877e+05   2.13e+04      8.815      0.000    1.46e+05    2.29e+05
zipcode_98027  1.788e+05   9834.456     18.176      0.000    1.59e+05    1.98e+05
zipcode_98028  1.138e+05   1.19e+04      9.593      0.000    9.05e+04    1.37e+05
zipcode_98029  1.927e+05   1.12e+04     17.159      0.000    1.71e+05    2.15e+05
zipcode_98030 -2.473e+04   1.23e+04     -2.004      0.045   -4.89e+04    -537.928
zipcode_98033  3.613e+05   9532.360     37.899      0.000    3.43e+05     3.8e+05
zipcode_98034  1.782e+05   8667.582     20.563      0.000    1.61e+05    1.95e+05
zipcode_98039   1.41e+06   2.66e+04     52.993      0.000    1.36e+06    1.46e+06
zipcode_98040  5.205e+05   1.18e+04     44.068      0.000    4.97e+05    5.44e+05
zipcode_98045  7.677e+04   1.31e+04      5.838      0.000     5.1e+04    1.03e+05
zipcode_98052  2.377e+05   8395.926     28.317   

In [31]:
# Test models

# Get a subset of the features and set to x and y

y = data_fin[['price']]
X = data_fin.drop(['price'], axis=1)

#from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#instantiate a linear regression object
lm = LinearRegression()

#fit the linear regression to the data
lm = lm.fit(X_train, y_train)

y_train_pred = lm.predict(X_train)

train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Training Root Mean Squared Error:' , train_rmse)

y_pred = lm.predict(X_test)

test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

print('Testing Root Mean Squared Error:' , test_rmse)


print('Training: ', train_rmse, "vs. Testing: ", test_rmse)

Training Root Mean Squared Error: 170468.41373626172
Testing Root Mean Squared Error: 166416.93399407945
Training:  170468.41373626172 vs. Testing:  166416.93399407945


In [29]:
# Test models

# Get a subset of the features and set to x and y

y = data_fin[['price']]
X = data_fin.drop(['price'], axis=1)

#from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

#from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
mse_test = np.sum((y_test-y_hat_test)**2)/len(y_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

# Pickle

Train Mean Squarred Error: price    2.919140e+10
dtype: float64
Test Mean Squarred Error: price    2.711806e+10
dtype: float64


In [32]:
## training the model with lasso

lasso = Lasso(alpha=0.01, normalize=False)

lasso.fit(X_train,y_train)

y_train_pred = lasso.predict(X_train)
y_pred = lasso.predict(X_test)

train_rmse = metrics.mean_absolute_error(y_train, y_train_pred)
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('Training Error: '+ str(train_rmse) )
print('Testing Error: '+ str(test_rmse) )

Training Error: 97946.93407744028
Testing Error: 166416.92882245287


In [ ]:
## training the model
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=.01, normalize=False)

lasso.fit(X_train,y_train)

y_train_pred_lasso = lasso.predict(X_train)
y_pred_lasso = lasso.predict(X_test)

train_rmse_lasso = metrics.mean_absolute_error(y_train, y_train_pred_lasso)
test_rmse_lasso = np.sqrt(metrics.mean_squared_error(y_test, y_pred_lasso))
print('Training Error: '+ str(train_rmse_lasso) )
print('Testing Error: '+ str(test_rmse_lasso) )

lasso_coef01 = pd.DataFrame(data=lasso.coef_).T
lasso_coef01.columns = X_train.columns
lasso_coef01 = lasso_coef01.T.sort_values(by=0).T
lasso_coef01.plot(kind='bar', title='Modal Coefficients', legend=False, figsize=(16,8))

In [ ]:
coefficient = lasso_coef01.T
coefficient[coefficient[0] != 0]

In [ ]:
#the difference between maximum and minimum: NRMSE=RMSEymax−ymin
print(train_rmse/(max(price)-min(price)))
print(test_rmse/(max(price)-min(price)))

In [ ]:
#the mean: NRMSE=RMSE¯
print(train_rmse/price.mean())
print(test_rmse/price.mean())

In [ ]:
y_train

In [ ]:
y_train_pred

In [ ]:
429900.0-527371.72972811

In [ ]:
285000.0 - 464392.93088471

In [ ]:
#displaying coefficients with lasso model

lasso_coef01 = pd.DataFrame(data=lasso.coef_).T
lasso_coef01.columns = X_train.columns
lasso_coef01 = lasso_coef01.T.sort_values(by=0).T
lasso_coef01.plot(kind='bar', title='Modal Coefficients', legend=False, figsize=(16,8))

In [ ]:
# checking for any coefficients that converge to 0

lasso_coef01.T